In [22]:
# Import the required library
import folium
from geopy.geocoders import Nominatim
import webbrowser
import requests
from requests.structures import CaseInsensitiveDict
from translate import Translator
import pandas as pd

translator = Translator(from_lang="autodetect",to_lang="en")

GEOAPIFY_API_KEY = "YOUR_GEOAPIFY_API_KEY"

city = str(input("Enter the name of the city that you plan to visit: "))
limit_results = 20

# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")
location = geolocator.geocode(city)

# Here we pass coordinates of Gfg and starting Zoom level = 12
my_map = folium.Map(location = [location.latitude, location.longitude], zoom_start = 12)

categories = ["tourism","tourism.attraction","tourism.sights","building.tourism","tourism.information"]

city_to_place_id_url = url = f"https://api.geoapify.com/v1/geocode/search?text={city}&apiKey={GEOAPIFY_API_KEY}"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

resp = requests.get(url, headers=headers)

print(resp.status_code)
result_city = resp.json()
city_id = result_city['features'][0]['properties']['place_id']

url = f"https://api.geoapify.com/v2/places?categories={', '.join(categories[:])}&filter=place:{city_id}&limit={limit_results}&apiKey={GEOAPIFY_API_KEY}"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

resp = requests.get(url, headers=headers)

print(resp.status_code)
results = resp.json()

data = []

for feature in results['features']:
    properties = feature['properties']
    datasource = properties['datasource']['raw']
    
    name = properties.get('name', f"Unknown - {properties.get('address_line1', 'name')}")
    address = properties.get('address_line2', 'N/A')
    
    description = datasource.get('description')
    if description:
        translated_description = translator.translate(description)
        if translated_description == "PLEASE SELECT TWO DISTINCT LANGUAGES":
            pass
        else:
            description = translated_description
    
    opening_hours = datasource.get('opening_hours', 'Unknown')
    
    coordinates = feature['geometry']['coordinates']

    ####
    if 'name' in feature['properties']:
        data.append({
        'Name': name,
        'Address': address,
        'Description': description,
        'Opening_hours': opening_hours,
        'Coordinates': coordinates
    })
    ####
    # data.append({
    #     'Name': name,
    #     'Address': address,
    #     'Description': description,
    #     'Opening_hours': opening_hours,
    #     'Coordinates': coordinates
    # })

df = pd.DataFrame(data)

# Create markers using dataframe rows
for _, row in df.iterrows():
    name = row['Name']
    address = row['Address']
    description = row['Description']
    opening_hours = row['Opening_hours']
    coordinates = row['Coordinates']
    
    # Create a marker and add it to the map
    marker = folium.Marker(
        location=[coordinates[1], coordinates[0]],
        icon=folium.Icon(color='blue')
    )
    
    # Create a custom HTML popup with desired width
    popup_html = f"""
    <div style="width: 300px;">
        <h3>{name}</h3>
        <p><strong>Address:</strong> {address}</p>
        <p><strong>Description:</strong> {description}</p>
        <p><strong>Opening Hours:</strong> {opening_hours}</p>
    </div>
    """
    
    # Add the custom HTML popup to the marker
    folium.Popup(popup_html, max_width=400).add_to(marker)
    
    # Add the marker to the map
    marker.add_to(my_map)

# Save the map as an HTML file
my_map.save("maps/map.html")

# Open the map in the default web browser
webbrowser.open("maps/map.html")

df

200
200


True